In [319]:
cd D:\Filed.Python

D:\Filed.Python


In [ ]:
### FOR the imports to work, change the path for config_file in FacebookDexter/BackgroundTasks/startup.py to absolute path 
# (for example: "D:/Filed.Python/FacebookDexter/BackGroundTasks/Config/Settings/app.settings.{env}.json"")

In [320]:
from FacebookDexter.Infrastructure.PersistanceLayer.DexterMongoRepository import DexterMongoRepository
from FacebookDexter.Infrastructure.Domain.Rules.RuleEvaluatorBuilder import RuleEvaluatorBuilder
from FacebookDexter.Infrastructure.Domain.Rules.RuleEvaluator import RuleEvaluator
from FacebookDexter.Infrastructure.Domain.Recommendations import RecommendationBuilder
from FacebookDexter.Infrastructure.Domain.LevelEnums import LevelEnum
from FacebookDexter.BackgroundTasks.startup import config, fixtures
from datetime import date, datetime, timedelta
from FacebookDexter.Infrastructure.Domain.Breakdowns import BreakdownEnum, BreakdownMetadata, ActionBreakdownEnum
from FacebookDexter.Infrastructure.Domain.Rules.Antecedent import Antecedent
from FacebookDexter.Infrastructure.Domain.LogicalOperatorEnum import LogicOperatorEnum
from FacebookDexter.Infrastructure.Domain.Metrics.Metric import Metric

In [321]:
today = date.today()
print (today)

2020-07-17


In [322]:
now = date.today() - timedelta(days=1)
three_days = timedelta(days=3)

three_days_ago = now-three_days

In [323]:
for bd in BreakdownEnum:
    print (bd)

BreakdownEnum.NONE
BreakdownEnum.AGE
BreakdownEnum.GENDER
BreakdownEnum.PLACEMENT
BreakdownEnum.DEVICE
BreakdownEnum.PLATFORM
BreakdownEnum.HOUR


In [324]:
class CustomMetricCalculator:
    def __init__(self):
        self.mongo_repostory = DexterMongoRepository(config = config.mongo)

        self.metric_to_formula_dictionary = {
            'campaign_budget': self.get_campaign_budget,
            'campaign_objective': self.get_campaign_objective,
            'metric_value': self.get_metric_value,
            'conversions_01_06': self.get_campaign_conversions_between_01_and_06_am,
            'conversion_rate_01_06': self.get_campaign_conversion_rate,            
            'amount_spent/budget': self.get_campaign_amount_spent_percentage_of_budget
        }
            
    
    def get_campaign_objective(self, campaign_id, date_start, date_stop):
        details = self.mongo_repostory.get_structure_details(campaign_id, LevelEnum.CAMPAIGN)
        return details['objective']

    def get_campaign_budget(self, campaign_id, date_start, date_stop):
        details = self.mongo_repostory.get_structure_details(campaign_id, LevelEnum.CAMPAIGN)
        if 'daily_budget' in details:            
            return int(details['daily_budget'])
        elif 'lifetime_budget' in details:
            # TODO: divide lifetime_budget by the number of days the campaign is supposed to run ??
            return int(details['lifetime_budget'])
        else:            
            ad_sets = self.mongo_repostory.get_adsets_by_campaign_id(campaign_id)
            campaign_daily_budget = 0
            campaign_lifetime_budget = 0
            for ad_set_id in ad_sets:
                details = self.mongo_repostory.get_structure_details(ad_set_id, LevelEnum.ADSET)
                if 'daily_budget' in details:                    
                    campaign_daily_budget += int(details['daily_budget'])
                elif 'lifetime_budget' in details:
                    campaign_lifetime_budget += details['lifetime_budget']
            if campaign_daily_budget > 0:
                return campaign_daily_budget
            elif campaign_lifetime_budget > 0:
                # TODO: divide lifetime_budget by the number of days the campaign is supposed to run ??
                return campaign_lifetime_budget
            else:
                raise ValueError('no campaign bugdet')

    def get_metric_value(self, campaign_id, date_start, date_stop, metrics, breakdown_metadata= None):
        if breakdown_metadata is None:
            breakdown_metadata = BreakdownMetadata()
            breakdown_metadata.breakdown = BreakdownEnum.NONE
            breakdown_metadata.action_breakdown = ActionBreakdownEnum.NONE
        return self.mongo_repostory.get_metrics_values(campaign_id, date_start, date_stop, metrics, LevelEnum.CAMPAIGN, breakdown_metadata)
    

    def get_campaign_metric_between_01_and_06_am(self, campaign_id, date_start, date_stop, metric):
        breakdown_metadata = BreakdownMetadata()
        breakdown_metadata.breakdown = BreakdownEnum.HOUR
        breakdown_metadata.breakdown_value = ['01:00:00 - 01:59:59', '02:00:00 - 02:59:59', '03:00:00 - 03:59:59', '04:00:00 - 04:59:59', '05:00:00 - 05:59:59']
        breakdown_metadata.action_breakdown = ActionBreakdownEnum.NONE        
        amounts = self.get_metric_value(campaign_id, date_start, date_stop, [metric], breakdown_metadata)
        campaign_amount = 0
        for amount in amounts:
            campaign_amount += amount[metric]        
        return amounts, campaign_amount    
    
    def get_campaign_amount_spent_percentage_of_budget(self, campaign_id, date_start, date_stop):
        budget = self.get_campaign_budget(campaign_id, date_start, date_stop)
        _ , amount_spent = self.get_campaign_metric_between_01_and_06_am(campaign_id, date_start, date_stop, 'amount_spent')
        print ('Budget: ', budget)
        print ('Amount Spent: ' , amount_spent)
        percentage = amount_spent / budget * 100
        print ('Perecentage: ', percentage)
        return percentage

    def get_campaign_conversion_rate(self, campaign_id, date_start, date_stop):               
        conversions, conversions_total = self.get_campaign_metric_between_01_and_06_am(campaign_id, date_start, date_stop, 'conversions')        
        link_clicks, link_clicks_total = self.get_campaign_metric_between_01_and_06_am(campaign_id, date_start, date_stop, 'link_clicks')        
        print ('Conversions: ', conversions_total)
        print ('Link Clicks:', link_clicks_total)
        conversion_rate = conversions_total/link_clicks_total * 100
        print ('Conversion Rate:', conversion_rate)
        return conversion_rate

    def get_campaign_conversions_between_01_and_06_am(self, campaign_id, date_start, date_stop):
        conversions, conversions_total = self.get_campaign_metric_between_01_and_06_am(campaign_id, date_start, date_stop, 'conversions')
        return conversions_total


    def get_metric_value_for_antecedent(self, antecedent: Antecedent, campaign_id, date_start, date_stop):        
        formula = self.metric_to_formula_dictionary.get(antecedent.metric.name)
        if formula is None:
            raise ValueError (f'Cant calculate {antecedent.metric.name}')        
        return formula(campaign_id, date_start, date_stop)

    def get_metric_value_for_template(self, metric, campaign_id, date_start, date_stop):        
        formula = self.metric_to_formula_dictionary.get(metric)
        if formula is None:
            raise ValueError (f'Cant calculate {antecedent.metric.name}')        
        return formula(campaign_id, date_start, date_stop)

calc = CustomMetricCalculator()

In [325]:
calc.get_campaign_metric_between_01_and_06_am('6161444487357', three_days_ago.strftime('%Y-%m-%d'), now.strftime('%Y-%m-%d'), 'conversions')

([{'_id': {'campaign_id': '6161444487357',
    'date_start': '2020-07-14',
    'hourly_stats_aggregated_by_advertiser_time_zone': '05:00:00 - 05:59:59'},
   'conversions': 0},
  {'_id': {'campaign_id': '6161444487357',
    'date_start': '2020-07-14',
    'hourly_stats_aggregated_by_advertiser_time_zone': '04:00:00 - 04:59:59'},
   'conversions': 0},
  {'_id': {'campaign_id': '6161444487357',
    'date_start': '2020-07-14',
    'hourly_stats_aggregated_by_advertiser_time_zone': '03:00:00 - 03:59:59'},
   'conversions': 0},
  {'_id': {'campaign_id': '6161444487357',
    'date_start': '2020-07-14',
    'hourly_stats_aggregated_by_advertiser_time_zone': '02:00:00 - 02:59:59'},
   'conversions': 0},
  {'_id': {'campaign_id': '6161444487357',
    'date_start': '2020-07-14',
    'hourly_stats_aggregated_by_advertiser_time_zone': '01:00:00 - 01:59:59'},
   'conversions': 0},
  {'_id': {'campaign_id': '6161444487357',
    'date_start': '2020-07-13',
    'hourly_stats_aggregated_by_advertiser_ti

In [326]:
recommendation_template_1 = 'Dexter noticed that between 01-06am you spend over {}% on the budget and have no conversions. Dexter suggests you to create the following rule: pause this campaign between 01-06am'
print (recommendation_template_1.format(5))
recommendation_template_2 = 'Dexter noticed that between 01-06am you spend over {}% on the budget and have a {}% conversion rate. Dexter suggests you to create the following rule: pause this campaign between 01-06am.'
print (recommendation_template_2.format(*[7, 0.4]))




Dexter noticed that between 01-06am you spend over 5% on the budget and have no conversions. Dexter suggests you to create the following rule: pause this campaign between 01-06am
Dexter noticed that between 01-06am you spend over 7% on the budget and have a 0.4% conversion rate. Dexter suggests you to create the following rule: pause this campaign between 01-06am.


In [327]:
class BranchStrategyNode:
        def __init__(self, false, true, rule: Antecedent, true_recommendation_template= None, true_recommendation_template_metrics= None,                                                        false_recommendation_template=None, false_recommendation_template_metrics=None):
            self.false = false
            self.true = true
            self.rule = rule
            self.true_recommendation_template = true_recommendation_template
            self.true_recommendation_template_metrics = true_recommendation_template_metrics
            self.false_recommendation_template = false_recommendation_template
            self.false_recommendation_template_metrics = false_recommendation_template_metrics
            

In [328]:
class NodeTraverser:
    def __init__(self, start_node: BranchStrategyNode, metric_calculator: CustomMetricCalculator, campaign_id, date_start, date_stop ):
        self.current_node = start_node
        self.metric_calculator = metric_calculator        
        self.recommendation = None
        self.campaign_id = campaign_id
        self.date_start = date_start
        self.date_stop = date_stop

    def traverse(self):
        while self.recommendation is None:            
            self.move_to_next_node()
        print (self.recommendation)
        return

    def move_to_next_node(self): 
        print ('Checking: ', self.current_node.rule.metric.name, self.current_node.rule.operator, self.current_node.rule.expected_value)           
        if self.evaluate_rule():
            self.move_to_true()
        else:
            self.move_to_false()

    def move_to_false(self):
        print ('moving to false')
        if self.current_node.false is not None:
            self.current_node = self.current_node.false
        else:
            if self.current_node.false_recommendation_template:
               self.recommendation = self.generate_recommendation(False)
            else:
               self.recommendation = {} 
            

    def move_to_true(self):
        print ('moving to true')
        if self.current_node.true is not None:
            self.current_node = self.current_node.true
        else:
            if self.current_node.true_recommendation_template:
                self.recommendation = self.generate_recommendation(True)
            else:
                self.recommendation = {}
            

    def evaluate_rule(self):
        print ('evaluating rule')
        return self.current_node.rule.evaluate(self.metric_calculator.get_metric_value_for_antecedent(self.current_node.rule,
                                                                                                      self.campaign_id,
                                                                                                      self.date_start,
                                                                                                      self.date_stop))
    def generate_recommendation(self, template: bool):
        if template:
            template_values = []
            for metric in self.current_node.true_recommendation_template_metrics:
                value = self.metric_calculator.get_metric_value_for_template(metric, self.campaign_id,
                                                                                self.date_start,
                                                                                self.date_stop)
                template_values.append(value) 
            return self.current_node.true_recommendation_template.format(*template_values)    
        else:
            template_values = []
            for metric in self.current_node.false_recommendation_template_metrics:
                value = self.metric_calculator.get_metric_value_for_template(metric, self.campaign_id,
                                                                                self.date_start,
                                                                                self.date_stop)
                template_values.append(value) 
            return self.current_node.false_recommendation_template.format(*template_values)    


In [329]:
result_rate_less_than_05 = Antecedent(aid=1, metric=Metric('conversion_rate_01_06'), expected_value=0.5, operator = LogicOperatorEnum.EQUAL_OR_LESS_THAN)

result_rate_less_than_05_node = BranchStrategyNode(None, None, result_rate_less_than_05, recommendation_template_2, ['amount_spent/budget', 'conversion_rate_01_06'])

conversion_are_0 = Antecedent(aid=1, metric=Metric('conversions_01_06'), expected_value=0, operator=LogicOperatorEnum.EQUALS)

conversions_are_0_node = BranchStrategyNode(result_rate_less_than_05_node, None, conversion_are_0, recommendation_template_1, ['amount_spent/budget'])

amount_spent_more_than_5_percent_of_budget = Antecedent(aid=1, metric=Metric('amount_spent/budget'), expected_value=1, operator=LogicOperatorEnum.EQUAL_OR_GREATER_THAN)

amount_spent_node = BranchStrategyNode(None, conversions_are_0_node, amount_spent_more_than_5_percent_of_budget)

objective_is_conversions = Antecedent(aid=1, metric=Metric('campaign_objective'), expected_value='CONVERSIONS', operator=LogicOperatorEnum.EQUALS)

objective_node = BranchStrategyNode(None, amount_spent_node, objective_is_conversions)


In [330]:
campaign_01_06_traverser = NodeTraverser(objective_node, calc, '6161444487357', str(three_days_ago), str(now))

In [331]:
campaign_01_06_traverser.traverse()

Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to true
Checking:  amount_spent/budget LogicOperatorEnum.EQUAL_OR_GREATER_THAN 1
evaluating rule
Budget:  2500
Amount Spent:  8.0
Perecentage:  0.32
moving to false
{}


In [332]:
campaign_01_06_traverser = NodeTraverser(objective_node, calc, '23845045569230327', str(three_days_ago), str(now))
campaign_01_06_traverser.traverse()

Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to true
Checking:  amount_spent/budget LogicOperatorEnum.EQUAL_OR_GREATER_THAN 1
evaluating rule
Budget:  20000
Amount Spent:  109.70848500000001
Perecentage:  0.5485424250000001
moving to false
{}


In [333]:
node_traverser = NodeTraverser(objective_node, calc, '23845014680770327', str(three_days_ago), str(now))
node_traverser.traverse()

Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to true
Checking:  amount_spent/budget LogicOperatorEnum.EQUAL_OR_GREATER_THAN 1
evaluating rule
Budget:  21000
Amount Spent:  427.405999
Perecentage:  2.035266661904762
moving to true
Checking:  conversions_01_06 LogicOperatorEnum.EQUALS 0
evaluating rule
moving to false
Checking:  conversion_rate_01_06 LogicOperatorEnum.EQUAL_OR_LESS_THAN 0.5
evaluating rule
Conversions:  10
Link Clicks: 315
Conversion Rate: 3.1746031746031744
moving to false
{}


In [334]:
node_traverser = NodeTraverser(objective_node, calc, '23845014680970327', str(three_days_ago), str(now))
node_traverser.traverse()

Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to true
Checking:  amount_spent/budget LogicOperatorEnum.EQUAL_OR_GREATER_THAN 1
evaluating rule
Budget:  45000
Amount Spent:  531.3616650000001
Perecentage:  1.1808037000000002
moving to true
Checking:  conversions_01_06 LogicOperatorEnum.EQUALS 0
evaluating rule
moving to false
Checking:  conversion_rate_01_06 LogicOperatorEnum.EQUAL_OR_LESS_THAN 0.5
evaluating rule
Conversions:  2
Link Clicks: 765
Conversion Rate: 0.261437908496732
moving to true
Budget:  45000
Amount Spent:  531.3616650000001
Perecentage:  1.1808037000000002
Conversions:  2
Link Clicks: 765
Conversion Rate: 0.261437908496732
Dexter noticed that between 01-06am you spend over 1.1808037000000002% on the budget and have a 0.261437908496732% conversion rate. Dexter suggests you to create the following rule: pause this campaign between 01-06am.


In [335]:
campaign_ids = ['23844799846470314',
'23844840722520314',
'23844968021030314',
'23845010064480327',
'23845010764870327',
'23845010933070327',
'23845014680770327',
'23845014680970327',
'23845014681170327',
'23845015989510327',
'23845015989680327',
'23845015989980327',
'23845021870210327',
'23845021870220327',
'23845021870580327',
'23845022312110327',
'23845022312130327',
'23845022312690327',
'23845022727750327',
'23845022727860327',
'23845022728150327',
'23845035665000327',
'23845035665100327',
'23845035665140327',
'23845035871170327',
'23845035871190327',
'23845035871650327',
'23845045569230327',
'23845139862580314',
'23845140264160314',
'23845140280760314',
'23845143657030327',
'23845162566680314',
'23845210185190314',
'23845237152400314',
'6161444487357',
'6163819700557',
'6195099940757',
'6195213681557']


for campaign_id in campaign_ids:
    print ('Checking campaign with id: ', campaign_id)
    node_traverser = NodeTraverser(objective_node, calc, campaign_id, str(three_days_ago), str(now))
    node_traverser.traverse()

Checking campaign with id:  23844799846470314
Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to true
Checking:  amount_spent/budget LogicOperatorEnum.EQUAL_OR_GREATER_THAN 1
evaluating rule
Budget:  2500
Amount Spent:  4.552497
Perecentage:  0.18209988
moving to false
{}
Checking campaign with id:  23844840722520314
Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to true
Checking:  amount_spent/budget LogicOperatorEnum.EQUAL_OR_GREATER_THAN 1
evaluating rule
Budget:  1500
Amount Spent:  3.0084199999999996
Perecentage:  0.2005613333333333
moving to false
{}
Checking campaign with id:  23844968021030314
Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to false
{}
Checking campaign with id:  23845010064480327
Checking:  campaign_objective LogicOperatorEnum.EQUALS CONVERSIONS
evaluating rule
moving to true
Checking:  amount_spent/budget LogicOperatorEnum.EQUAL_OR_GR